In [2]:
import pandas as pd 
import numpy as np 

In [47]:
df = pd.read_csv("ola.csv", index_col = 0)
df.head(10)

,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,12/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
5,12/01/19,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
6,01/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
7,02/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,0,1
8,03/01/20,4,43.0,0.0,C13,2,65603,12/07/19,NaN,2,2,350000,1
9,04/01/20,4,43.0,0.0,C13,2,65603,12/07/19,27/04/20,2,2,0,1


### Let's Understand the DATA --- Initial Guesses, I am writing down

1) The Dateofjoining column is date when the driver joined the OLA ecosystem.
2) The first column MMM-YY seems the date when the driver utilized OLA platform and offered service - so i can use how long it took him to start using OLA after joining the platform.
3) Observe carefully, for each driver (having a unique Driver_ID) there are multiple rows, indicating his number of trips. Each column is a signal that he was active on that day and gave service.
4) LastWorkingDate column indicates that if the value is NaN, he is still part of OLA network. If there is a value of date present there, it indicates he deleted his account from OLA platform.
5) The data given to us is for year 2019 and 2020. So i can also know that which drivers are still active in 2020.
6) Gender column might be skewed. I have to see it. 
7) More elder and more younger drivers might have random behaviour.
8) City might affect churn behaviour of a driver because it can be case that there are no people in that city interested in booking cabs and that too with a OLA.
9) Education level is also an factor because maybe some drivers would have joined OLA because of unemployment? 
10) Income is also a key factor. Here it seems income is represented in some form of encoding, would have to look into it.



### Here is the information provided by OLA

1) Gender : Gender of the driver – Male : 0, Female: 1
2) MMMM-YY : Reporting Date (Monthly)
3) Education_Level : Education level – 0 for 10+ ,1 for 12+ ,2 for graduate
4) Income : Monthly average Income of the driver
5) The total business value acquired by the driver in a month (negative business indicates cancellation/refund or car EMI adjustments)
6) Quarterly Rating : Quarterly rating of the driver: 1,2,3,4,5 (higher is better)




#### From ChatGPT: 

Total Business Value (TBV): This is the sum total of all financial gains and losses associated with the driver's operations within a given month.

Positive Value: If the TBV is positive, it indicates the total earnings and profits generated by the driver during the month. This can include revenue from completed rides, tips, and any additional income.

Negative Value: If the TBV is negative, it suggests that the driver experienced losses or incurred expenses that outweighed their earnings. Possible reasons for a negative TBV include cancellations or refunds, adjustments for car loan EMI (Equated Monthly Installment), or other costs that exceeded the driver's income.

Cancellations or refunds might occur if a driver cancels a ride or if a passenger requests and receives a refund. Car EMI adjustments suggest that a portion of the negative business value could be attributed to adjustments related to the driver's car loan EMI.



##### I can hypothise that a person having taken a loan from OLA, have higher chance of Quitting, because he might not be able to generate enough income to payback the EMI and book a profit

In [48]:
# let's compress the rows for each driver

# but first let's store all columns in a list

cols = df.columns

In [49]:
agg = {'MMM-YY': 'last',
 'Age': 'last',
 'Gender': 'last',
 'City': 'last',
 'Education_Level': 'last',
 'Income': 'last',
 'Dateofjoining': 'last',
 'LastWorkingDate': 'last',
 'Joining Designation': 'last',
 'Grade': 'last',
 'Total Business Value': 'sum',
 'Quarterly Rating': 'last'}

In [50]:
new_df = df.groupby('Driver_ID').aggregate(agg).reset_index()
new_df.head()

,Driver_ID,MMM-YY,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,03/01/19,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,1715580,2
1,2,12/01/20,31.0,0.0,C7,2,67016,11/06/20,None,2,2,0,1
2,4,04/01/20,43.0,0.0,C13,2,65603,12/07/19,27/04/20,2,2,350000,1
3,5,03/01/19,29.0,0.0,C9,0,46368,01/09/19,03/07/19,1,1,120360,1
4,6,12/01/20,31.0,1.0,C11,1,78728,31/07/20,None,3,3,1265000,2


In [59]:
new_df['Dateofjoining'] = pd.to_datetime(new_df['Dateofjoining'], format='%d/%m/%y')
new_df['LastWorkingDate'] = pd.to_datetime(new_df['LastWorkingDate'], format='%d/%m/%y')
new_df['MMM-YY'] = pd.to_datetime(new_df['MMM-YY'], format='%m/%d/%y')

# Always be careful and observe data to find what is the date format given.

In [65]:
new_df = new_df.rename(columns = {'MMM-YY':'LastReport'})
new_df.head()

,Driver_ID,LastReport,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,2019-03-01,28.0,0.0,C23,2,57387,2018-12-24,2019-11-03,1,1,1715580,2
1,2,2020-12-01,31.0,0.0,C7,2,67016,2020-06-11,NaT,2,2,0,1
2,4,2020-04-01,43.0,0.0,C13,2,65603,2019-07-12,2020-04-27,2,2,350000,1
3,5,2019-03-01,29.0,0.0,C9,0,46368,2019-09-01,2019-07-03,1,1,120360,1
4,6,2020-12-01,31.0,1.0,C11,1,78728,2020-07-31,NaT,3,3,1265000,2


#### Further Information



In [67]:
df['Dateofjoining'] = pd.to_datetime(df['Dateofjoining'], format='%d/%m/%y')
df['LastWorkingDate'] = pd.to_datetime(df['LastWorkingDate'], format='%d/%m/%y')
df['MMM-YY'] = pd.to_datetime(df['MMM-YY'], format='%m/%d/%y')

# Always be careful and observe data to find what is the date format given.

In [145]:
# this data tells about when driver joined the OLA network and when he Came to first Report for duty.
# the report column in MMM-YY
driver_join_data = df.groupby('Driver_ID').agg({'MMM-YY': 'first', 'Dateofjoining': 'first', 'LastWorkingDate':'last'})
driver_join_data = driver_join_data.rename(columns = {"MMM-YY":"FirstReport"})
driver_join_data.head()

,FirstReport,Dateofjoining,LastWorkingDate
Driver_ID,,,
1,2019-01-01,2018-12-24,2019-11-03
2,2020-11-01,2020-06-11,NaT
4,2019-12-01,2019-07-12,2020-04-27
5,2019-01-01,2019-09-01,2019-07-03
6,2020-08-01,2020-07-31,NaT


In [146]:
driver_join_data['Firstreport_leadtime'] = (driver_join_data['FirstReport'] - driver_join_data['Dateofjoining']).dt.days
driver_join_data.head()

,FirstReport,Dateofjoining,LastWorkingDate,Firstreport_leadtime
Driver_ID,,,,
1,2019-01-01,2018-12-24,2019-11-03,8
2,2020-11-01,2020-06-11,NaT,143
4,2019-12-01,2019-07-12,2020-04-27,142
5,2019-01-01,2019-09-01,2019-07-03,-243
6,2020-08-01,2020-07-31,NaT,1


### As we can see that it is possible that drivers who have first reported (mostly for inquiry but are given id by system) and later joined OLA online platform have negative Firstreport_leadtime
### For drivers who have online registered themselves but not reported in office, they have positive Firstreport_leadtime


In [147]:
# so to consider the total_service_days of a driver on ola platform, i will consider the first_report_date and last_working_date
driver_join_data['service_days'] = (driver_join_data['LastWorkingDate'] - driver_join_data['FirstReport']).dt.days
driver_join_data.head()

,FirstReport,Dateofjoining,LastWorkingDate,Firstreport_leadtime,service_days
Driver_ID,,,,,
1,2019-01-01,2018-12-24,2019-11-03,8,306.0
2,2020-11-01,2020-06-11,NaT,143,NaN
4,2019-12-01,2019-07-12,2020-04-27,142,148.0
5,2019-01-01,2019-09-01,2019-07-03,-243,183.0
6,2020-08-01,2020-07-31,NaT,1,NaN


#### It is logical that for those drivers who have LastWorkingDate as Not available - They are still connected to OLA platform. So to consider their serive_days i would consider it with respect to a reference date i.e. 31st december 2020. (Remember, we have data for 2019 & 2020 only)

In [148]:
driver_join_data['LastWorkingDate'] = driver_join_data['LastWorkingDate'].fillna(pd.to_datetime('2020-12-31'))
driver_join_data['service_days'] = (driver_join_data['LastWorkingDate'] - driver_join_data['FirstReport']).dt.days
driver_join_data.head()

,FirstReport,Dateofjoining,LastWorkingDate,Firstreport_leadtime,service_days
Driver_ID,,,,,
1,2019-01-01,2018-12-24,2019-11-03,8,306
2,2020-11-01,2020-06-11,2020-12-31,143,60
4,2019-12-01,2019-07-12,2020-04-27,142,148
5,2019-01-01,2019-09-01,2019-07-03,-243,183
6,2020-08-01,2020-07-31,2020-12-31,1,152


In [150]:
driver_join_data = driver_join_data.reset_index()

In [157]:
# Now let's join the Firstreport_leadtime and service_days data to new_df
new_df['churn'] = new_df['LastWorkingDate'].notna().astype(int).tolist()

In [162]:
new_df = new_df.merge(driver_join_data[['Driver_ID', 'Firstreport_leadtime', 'service_days']], on = 'Driver_ID', how = 'left')
new_df = new_df.drop(columns = ['LastReport', 'Dateofjoining', 'LastWorkingDate'] )
new_df.head()

,Driver_ID,Age,Gender,City,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating,churn,Firstreport_leadtime,service_days
0,1,28.0,0.0,C23,2,57387,1,1,1715580,2,1,8,306
1,2,31.0,0.0,C7,2,67016,2,2,0,1,0,143,60
2,4,43.0,0.0,C13,2,65603,2,2,350000,1,1,142,148
3,5,29.0,0.0,C9,0,46368,1,1,120360,1,1,-243,183
4,6,31.0,1.0,C11,1,78728,3,3,1265000,2,0,1,152
